In [2]:
import xarray as xr
import fsspec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client, progress
from pathlib import Path 

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\xarray\core\merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:50558 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.06 GB


In [4]:
url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr'

In [5]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)

Wall time: 10.6 s


In [6]:
ds

<xarray.Dataset>
Dimensions:   (time: 367440, x: 4608, y: 3840)
Coordinates:
  * time      (time) datetime64[ns] 1979-02-01 ... 2020-12-31T23:00:00
  * x         (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y         (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    RAINRATE  (time, y, x) float32 dask.array<shape=(367440, 3840, 4608), chunksize=(672, 350, 350)>
    crs       |S1 ...
Attributes:
    NWM_version_number:   v2.1
    model_configuration:  AORC
    model_output_type:    forcing

In [7]:
ds.RAINRATE


<xarray.DataArray 'RAINRATE' (time: 367440, y: 3840, x: 4608)>
dask.array<shape=(367440, 3840, 4608), dtype=float32, chunksize=(672, 350, 350)>
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01 ... 2020-12-31T23:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    cell_methods:    time: mean
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Surface Precipitation Rate
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    standard_name:   precipitation_flux
    units:           mm s^-1

In [17]:
ds.RAINRATE.esri_pe_string

'PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DATUM["D_Sphere",SPHEROID["Sphere",6370000.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-97.0],PARAMETER["standard_parallel_1",30.0],PARAMETER["standard_parallel_2",60.0],PARAMETER["latitude_of_origin",40.0],UNIT["Meter",1.0]];-35691800 -29075200 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'

In [18]:
## input the coordinates {y1:x1, y2:x2} or {lat1:long1, lat2:long2} format
Coordinates = {-1131207.186: 166599.7097} # Hobby Airport
##Coordinates = {-1115315.2193: 81203.1552,-1115178.9781: 73202.0731,-1114745.2073: 85011.1184,-1114823.3322: 77198.5283,-1110537.0666: 81155.3195,-1110497.0396: 85059.5592,-1110576.9827: 77065.1435,-1110111.0867: 69474.9337,-1110093.8945: 92771.3761,-1110030.4448: 88959.1182,-1110080.6105: 73341.9814,-1106436.4457: 77025.7806,-1106289.4588: 81205.7142,-1106220.0463: 65409.2229,-1105896.1206: 88356.2480,-1105632.8797: 92906.1487,-1105722.5089: 84730.7200,-1105731.1558: 72930.9845,-1102110.4083: 84971.5581,-1101497.2647: 92487.2548,-1100795.8328: 88764.9134}

In [19]:

t=0
header = ''
for k, v in Coordinates.items():
    cord = str(k) + ',' + str(v)
    if (t==0):
        a = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df = a.to_pandas()
        df =df.to_frame()
        df.rename(columns={0: cord}, inplace=True)
        
    else:
        a1 = ds.RAINRATE.sel(y = k, x= v, method="nearest" )
        df1 = a1.to_pandas()
        df1 =df1.to_frame()
        df1.rename(columns={0: cord}, inplace=True)
        
        df= pd.merge(df, df1, suffixes=(False, False), left_index=True, right_index=True )
    t+=1
    

C:\Users\ahmed\AppData\Local\Continuum\anaconda3\lib\site-packages\xarray\core\dataarray.py:1619: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}


In [20]:
df

,"-1131207.186,166599.7097"
time,
1979-02-01 00:00:00,0.000000
1979-02-01 01:00:00,0.000000
1979-02-01 02:00:00,0.000000
1979-02-01 03:00:00,0.000000
1979-02-01 04:00:00,0.000000
...,...
2020-12-31 19:00:00,0.000333
2020-12-31 20:00:00,0.000389
2020-12-31 21:00:00,0.000194


In [21]:
filepath = Path('C:/Users/ahmed/Documents/pcp_hobbyairport.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  